In [95]:
import numpy as np
import neuroglancer
from caveclient import CAVEclient

In [20]:
cave_client = CAVEclient(datastack_name="v1dd", server_address="https://globalv1.em.brain.allentech.org")
cave_client.info.image_source()

'graphene://https://api.em.brain.allentech.org/proxy/v1dd/image/aligned_image'

In [156]:
cave_client.state.get_state_json(4623266593374208)

{'layers': [{'source': 'graphene://https://api.em.brain.allentech.org/proxy/v1dd/image/aligned_image',
   'type': 'image',
   'blend': 'default',
   'shader': '#uicontrol float black slider(min=0, max=1, default=0.35)\n#uicontrol float white slider(min=0, max=1, default=0.65)\nfloat rescale(float value) {\n  return (value - black) / (white - black);\n}\nvoid main() {\n  float val = toNormalized(getDataValue());\n  if (val < black) {\n    emitRGB(vec3(0,0,0));\n  } else if (val > white) {\n    emitRGB(vec3(1.0, 1.0, 1.0));\n  } else {\n    emitGrayscale(rescale(val));\n  }\n}\n',
   'shaderControls': {},
   'name': 'EM-img'},
  {'source': 'graphene://https://api.em.brain.allentech.org/segmentation/table/aibs_v1dd',
   'type': 'image',
   'blend': 'default',
   'shaderControls': {},
   'name': 'EM-seg'},
  {'type': 'annotation',
   'annotations': [],
   'annotationTags': [],
   'voxelSize': [9, 9, 45],
   'bracketShortcutsShowSegmentation': True,
   'annotationSelectionShowsSegmentation'

In [64]:
import caveclient
caveclient.jsonservice.JSONServiceV1?

Init signature:
caveclient.jsonservice.JSONServiceV1(
    server_address,
    auth_header,
    api_version,
    endpoints,
    server_name,
    ngl_url,
    max_retries=None,
    pool_maxsize=None,
    pool_block=None,
    over_client=None,
)
Docstring:      <no docstring>
File:           /opt/anaconda3/envs/allen_v1dd/lib/python3.10/site-packages/caveclient/jsonservice.py
Type:           type
Subclasses:     

In [87]:
v1dd_transform_vx()

Transformation Sequence:
	Scale by [[ 9  9 45]]
	Rotate with params ('x', -15.526727064925824) and {'degrees': True}
	Rotate with params ('y', 0.0) and {'degrees': True}
	Rotate with params ('z', -0.28519951516860176) and {'degrees': True}
	Translate by [      0.         -385268.43177721       0.        ]
	Scale by [[0.001 0.001 0.001]]

In [89]:
from scipy.spatial.transform import Rotation as R

In [103]:
def _rotation_from_up_vector(up):
    rot, _ = R.align_vectors(np.array([[0, 1, 0]]), [up])
    return rot

up = np.array([-0.00497765, 0.96349375, 0.26768454])
rot = _rotation_from_up_vector(up)
ROTATION_QUAT = list(rot.as_quat())

/var/folders/8l/944wpj6s7sq8gcv57wcd3q6h0000gn/T/ipykernel_30602/3512578532.py:2: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  rot, _ = R.align_vectors(np.array([[0, 1, 0]]), [up])


In [108]:
ROTATION_MATRIX_4x4 = rot.as_matrix().tolist()
ROTATION_MATRIX_4x4[0].append(0)
ROTATION_MATRIX_4x4[1].append(0)
ROTATION_MATRIX_4x4[2].append(0)
ROTATION_MATRIX_4x4.append([0, 0, 0, 1])

[[0.9999876114234538, 0.0047959940982429655, 0.0013324564627750658, 0],
 [-0.00497765000934903, 0.9634937518096357, 0.2676845405027656, 0],
 [1.0842021724855044e-19, -0.26768785677426976, 0.9635056882736075, 0],
 [0, 0, 0, 1]]

In [160]:
BW_SLIDER = """
#uicontrol float black slider(min=0, max=1)
#uicontrol float white slider(min=0, max=1)
float rescale(float value) {
  return (value - black) / (white - black);
}
void main() {
  float val = toNormalized(getDataValue());
  if (val < black) {
    emitRGB(vec3(0,0,0));
  } else if (val > white) {
    emitRGB(vec3(1.0, 1.0, 1.0));
  } else {
    emitGrayscale(rescale(val));
  }
}
""".strip()

In [194]:
EM_IMAGE_LAYER = dict(
    name = "EM-img",
    type = "image",
    source = cave_client.info.image_source(),
    
    blend = "default",
    shader = BW_SLIDER,
    shaderControls = dict(
        black = 0.35,
        white = 0.65
    ),

    # transform = ROTATION_MATRIX_4x4
)

EM_SEGMENTATION_LAYER = dict(
    name = "EM-seg",
    type = "segmentation_with_graph",
    source = cave_client.info.segmentation_source(),
    
    selectedAlpha = 0.3,
    skeletonRendering = {
        "mode2d": "lines_and_points",
        "mode3d": "lines"
    },
    graphOperationMarker = [
        {
            "annotations": [],
            "tags": []
        },
        {
            "annotations": [],
            "tags": []
        }
    ],
    pathFinder = {
        "color": "#ffff00",
        "pathObject": {
            "annotationPath": {
                "annotations": [],
                "tags": []
            },
            "hasPath": False
        }
    },
    
    visible = False
)

In [193]:
state

{'layers': [{'name': 'EM-img',
   'type': 'image',
   'source': 'graphene://https://api.em.brain.allentech.org/proxy/v1dd/image/aligned_image',
   'blend': 'default',
   'shader': '#uicontrol float black slider(min=0, max=1)\n#uicontrol float white slider(min=0, max=1)\nfloat rescale(float value) {\n  return (value - black) / (white - black);\n}\nvoid main() {\n  float val = toNormalized(getDataValue());\n  if (val < black) {\n    emitRGB(vec3(0,0,0));\n  } else if (val > white) {\n    emitRGB(vec3(1.0, 1.0, 1.0));\n  } else {\n    emitGrayscale(rescale(val));\n  }\n}',
   'shaderControls': {'black': 0.35, 'white': 0.65}},
  {'name': 'EM-seg',
   'type': 'segmentation_with_graph',
   'source': 'graphene://https://api.em.brain.allentech.org/segmentation/table/aibs_v1dd',
   'selectedAlpha': 0.3,
   'skeletonRendering': {'mode2d': 'lines_and_points', 'mode3d': 'lines'},
   'graphOperationMarker': [{'annotations': [], 'tags': []},
    {'annotations': [], 'tags': []}],
   'pathFinder': {'c

In [199]:
def build_anno_layer(name="anno"):
    return dict(
        name = name,
        type = "annotation",
        annotations = [],
        annotationTags = [],
        voxelSize = cave_client.info.viewer_resolution().tolist(),
        bracketShortcutsShowSegmentation = True,
        annotationSelectionShowsSegmentation = True,
    )

def build_state_json(anno_layer, viewer_coords_vx, casey_rotate=False):
    state = dict(
        layers = [EM_IMAGE_LAYER, EM_SEGMENTATION_LAYER, anno_layer],
        navigation = dict(
            pose = dict(
                position = dict(
                    voxelSize = cave_client.info.viewer_resolution().tolist(),
                    voxelCoordinates = viewer_coords_vx
                )
            ),
            zoomFactor = 400
        ),
        showSlices = False,
        jsonStateServer = cave_client.state.state_service_endpoint,
        layout = "xz-3d"
    )
    return state
    

In [200]:
viewer_coords_vx = [107669.515625, 79662.5625, 8512.7724609375]

anno_layer = build_anno_layer()
state = build_state_json(anno_layer, viewer_coords_vx)
state_id = cave_client.state.upload_state_json(state)
url = cave_client.state.build_neuroglancer_url(state_id)

url

'https://neuroglancer.neuvue.io/?json_url=https://globalv1.em.brain.allentech.org/nglstate/api/v1/5029033628663808'

In [183]:
cave_client.state.build_neuroglancer_url?

Signature: cave_client.state.build_neuroglancer_url(state_id, ngl_url=None)
Docstring:
Build a URL for a Neuroglancer deployment that will automatically retrieve specified state.
If the datastack is specified, this is prepopulated from the info file field "viewer_site".
If no ngl_url is specified in either the function or the client, only the JSON state url is returned.

Parameters
----------
state_id : int
    State id to retrieve
ngl_url : str
    Base url of a neuroglancer deployment. If None, defaults to the value for the datastack or the client.
    If no value is found, only the URL to the JSON state is returned.

Returns
-------
str
    The full URL requested
File:      /opt/anaconda3/envs/allen_v1dd/lib/python3.10/site-packages/caveclient/jsonservice.py
Type:      method

In [110]:
from allen_v1dd.client import EMClient

em_client = EMClient()

In [112]:
coreg_table = em_client.get_coregistration_table()
coreg_table.head()

,index,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,...,ophys_volume,ophys_session_id,ophys_plane,ophys_roi,roi,position_microns,axon_proof_status,axon_proof_complete,dendrite_proof_status,dendrite_proof_complete
0,0,276266,2022-02-04 06:21:16.129002+00:00,t,377.762618,91491531315515547,864691132827688641,1,2023-04-14 22:08:40.446962+00:00,t,...,3,M409828_13,0,117,M409828_13_0_117,"[778.7521791075498, 227.97907876189072, 29.657...",submitted,True,not_started,True
1,1,276140,2022-02-04 06:30:31.896172+00:00,t,321.738753,92547406008789124,864691132728579766,2,2023-04-14 22:08:40.447864+00:00,t,...,3,M409828_13,0,75,M409828_13_0_75,"[816.1188928555765, 236.2662880987557, 17.5425...",submitted,True,submitted,True
2,2,290027,2022-02-04 06:25:08.200797+00:00,t,309.625983,94237492550599227,864691132727393198,3,2023-04-14 22:08:40.448713+00:00,t,...,3,M409828_13,0,26,M409828_13_0_26,"[874.2243220704512, 248.26178219838565, -104.6...",not_started,False,not_started,False
3,3,365414,2022-02-04 06:31:02.914671+00:00,t,363.636744,95855767575844611,864691132782526604,4,2023-04-14 22:08:40.449580+00:00,t,...,3,M409828_13,0,163,M409828_13_0_163,"[933.6856808451048, 252.00293174161231, -63.72...",not_started,False,not_started,False
4,4,365704,2022-02-04 06:21:20.137086+00:00,t,380.395452,97192636343210349,864691132715818816,5,2023-04-14 22:08:40.450423+00:00,t,...,3,M409828_13,0,31,M409828_13_0_31,"[979.6986711301596, 251.2837669970616, -46.357...",not_started,False,not_started,False


In [114]:
neuroglancer.PointAnnotation

Init signature: neuroglancer.PointAnnotation(*args, **kwargs)
Docstring:      <no docstring>
File:           /opt/anaconda3/envs/allen_v1dd/lib/python3.10/site-packages/neuroglancer/viewer_state.py
Type:           type
Subclasses:     

In [ ]:
state_id = cave_client.state.upload_state_json(state)

In [136]:
from nglui.statebuilder import *

In [119]:
anno_layer = AnnotationLayerConfig(name="anno")

anno_layer?

Type:        AnnotationLayerConfig
String form: <nglui.statebuilder.layers.AnnotationLayerConfig object at 0x7f887a7a0400>
File:        /opt/anaconda3/envs/allen_v1dd/lib/python3.10/site-packages/nglui/statebuilder/layers.py
Docstring:  
Configuration class for annotation layers

Parameters
----------
name : str, optional
    Layer name. By default, 'annos'
color : str, optional
    Hex color code with an initial #. By default, None
linked_segmentation_layer : str, optional
    Name of a linked segmentation layer for selected ids. By default, None
mapping_rules : PointMapper, LineMapper, SphereMapper or list, optional
    One rule or a list of rules mapping data to annotations. By default, []
array_data : bool, optional
    If True, allows simple mapping where one or more arrays are passed instead of a dataframe.
    Only allows basic annotation creation, no tags, linked segmentations, or other rich features.
tags : list, optional
    List of tags for the layer.
active : bool, optional

In [163]:
AnnotationLayerConfig?

Init signature:
AnnotationLayerConfig(
    name=None,
    color=None,
    linked_segmentation_layer=None,
    mapping_rules=[],
    array_data=False,
    tags=None,
    active=True,
    filter_by_segmentation=False,
    brackets_show_segmentation=True,
    selection_shows_segmentation=True,
    filter_query=None,
    data_resolution=None,
)
Docstring:     
Configuration class for annotation layers

Parameters
----------
name : str, optional
    Layer name. By default, 'annos'
color : str, optional
    Hex color code with an initial #. By default, None
linked_segmentation_layer : str, optional
    Name of a linked segmentation layer for selected ids. By default, None
mapping_rules : PointMapper, LineMapper, SphereMapper or list, optional
    One rule or a list of rules mapping data to annotations. By default, []
array_data : bool, optional
    If True, allows simple mapping where one or more arrays are passed instead of a dataframe.
    Only allows basic annotation creation, no tags, li

In [157]:
ImageLayerConfig?

Init signature:
ImageLayerConfig(
    source,
    name=None,
    active=False,
    contrast_controls=False,
    black=0.0,
    white=1.0,
)
Docstring:     
Image layer configuration class.

This provides the rules for setting up an image layer in neuroglancer.

Parameters
----------
source : str
    Cloudpath to an image source
name : str, optional
    Name of the image layer. By default, 'img'.
active : bool, optional
    If True, makes the layer active in neuroglancer. Default is False.
contrast_controls : bool, optional
    If True, gives the layer a user-controllable brightness and contrast shader. Default is False.
black : float, optional
    If contrast_controls is True, sets the default black level. Default is 0.0.
white : float, optional
    If contrast_controls is True, sets the default white level. Default is 1.0.
File:           /opt/anaconda3/envs/allen_v1dd/lib/python3.10/site-packages/nglui/statebuilder/layers.py
Type:           type
Subclasses:     

In [158]:
em_image_layer = ImageLayerConfig(
    name = "EM-img",
    source = cave_client.info.image_source(),
    contrast_controls = True,
    black = 0.35,
    white = 0.65,
)

em_segmentation_layer = ImageLayerConfig(
    name = "EM-seg",
    source = cave_client.info.segmentation_source()
)

sb = StateBuilder(
    layers = [em_image_layer, em_segmentation_layer, anno_layer],
    state_server = "https://global.em.brain.allentech.org/nglstate/api/v1/post",
    view_kws = dict(
        layout = "xz-3d"
    )
)

sb.render_state(return_as="url", url_prefix=cave_client.info.viewer_site(), base_state=BASE_STATE)

'https://neuroglancer.neuvue.io/#!%7B%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B9,9,45%5D%7D,%22orientation%22:%5B-0.13508161545914635,0.00033619658472511314,-0.0024660211129508766,0.9908313493610678%5D%7D,%22zoomFactor%22:2.0%7D,%22jsonStateServer%22:%22https://global.em.brain.allentech.org/nglstate/api/v1/post%22,%22showSlices%22:false,%22layout%22:%22xz-3d%22,%22perspectiveZoom%22:2000.0,%22layers%22:%5B%7B%22type%22:%22image%22,%22source%22:%22graphene://https://api.em.brain.allentech.org/proxy/v1dd/image/aligned_image%22,%22name%22:%22EM-img%22,%22shader%22:%22#uicontrol%20float%20black%20slider(min=0,%20max=1,%20default=0.35)%5Cn#uicontrol%20float%20white%20slider(min=0,%20max=1,%20default=0.65)%5Cnfloat%20rescale(float%20value)%20%7B%5Cn%20%20return%20(value%20-%20black)%20/%20(white%20-%20black);%5Cn%7D%5Cnvoid%20main()%20%7B%5Cn%20%20float%20val%20=%20toNormalized(getDataValue());%5Cn%20%20if%20(val%20%3C%20black)%20%7B%5Cn%20%20%20%20emitRGB(vec3(0